In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
import tensorflow as tf

In [ ]:
img_color = cv2.imread('sudoku.jpeg', 1)
img_gray = cv2.imread('sudoku.jpeg', 0)

image_area = img_gray.shape[0] * img_gray.shape[1]

img = cv2.bilateralFilter(img_gray, 9, 75, 75)
img = cv2.adaptiveThreshold(img_gray,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY,11,2)

In [ ]:
im2, contours, hierarchy = cv2.findContours(img, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

biggest_contour = None
max_area = 0
for cnt in contours:
    area = cv2.contourArea(cnt)
    if area > 100 and area > max_area:
        perimeter = cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, 0.02*perimeter, True)
        if len(approx) == 4 and area/image_area < 0.95:
            biggest_contour = approx
            max_area = area
            
            
cv2.drawContours(img_color, [biggest_contour], 0, (0, 0, 255), 3)

plt.imshow(img_color)
plt.axis("off")
plt.show()

In [ ]:
vertices = np.zeros([4, 2], dtype=np.float32)
contour_sum = biggest_contour.sum(axis = 2)
vertices[0] = biggest_contour[contour_sum.argmin(), 0, :]
vertices[2] = biggest_contour[contour_sum.argmax(), 0, :]
l = [0, 1, 2, 3]
l.remove(contour_sum.argmin())
l.remove(contour_sum.argmax())
if biggest_contour[l[0], 0, 0] < biggest_contour[l[1], 0, 0]:
    vertices[1] = biggest_contour[l[0], 0, :]
    vertices[3] = biggest_contour[l[1], 0, :]
else:
    vertices[1] = biggest_contour[l[1], 0, :]
    vertices[3] = biggest_contour[l[0], 0, :]

print(vertices)
mapped_vertices = np.array([[0, 0], [0, 449], [449, 449], [449, 0]], dtype=np.float32)

In [ ]:
M = cv2.getPerspectiveTransform(vertices, mapped_vertices)
sudoku_img = cv2.warpPerspective(img_gray, M, (450, 450))

plt.imshow(sudoku_img, cmap='gray')
plt.axis('off')
plt.show()

In [ ]:
digits = []

for i in range(9):
    for j in range(9):
        digit = [50*i + 4 : 50*(i+1) - 4, 50*j + 4: 50*(j+1) - 4]
        # kernel = np.ones((2, 2),np.uint8)
        # digit = cv2.erode(digit, kernel, iterations = 1)
        digits.append(digit)
        plt.subplot(9, 9, i*9 + j + 1)
        plt.imshow(digit, cmap='gray')
        plt.axis('off')
plt.show()

digits = numpy.array(digits)

In [ ]:
def CNNClassifier(images):
    with tf.name_scope('input'):
        x = tf.placeholder(tf.float32, name = "x")
        x_image = tf.reshape(x, [-1, 42, 42, 1])

    with tf.name_scope('conv1'):
        w1 = tf.Variable(tf.truncated_normal([5, 5, 1, 32], stddev = 0.1), name="w1")
        b1 = tf.Variable(tf.truncated_normal([32], stddev = 0.1), name="b1")
        conv1 = tf.nn.relu(tf.add(tf.nn.conv2d(x_image, w1, strides = [1, 1, 1, 1], padding='SAME'), b1))

    with tf.name_scope('pool1'):
        pool1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

    with tf.name_scope('conv2'):
        w2 = tf.Variable(tf.truncated_normal([5, 5, 32, 64], stddev = 0.1), name="w2")
        b2 = tf.Variable(tf.truncated_normal([64], stddev = 0.1), name="b2")
        conv2 = tf.nn.relu(tf.add(tf.nn.conv2d(pool1, w2, strides = [1, 1, 1, 1], padding = 'SAME'), b2))

    with tf.name_scope('pool2'):
        pool2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

    with tf.name_scope('fc1'):
        w_fc1 = tf.Variable(tf.truncated_normal([11*11*64, 1536], stddev = 0.1), name="w_fc1")
        b_fc1 = tf.Variable(tf.truncated_normal([1536], stddev = 0.1), name = "b_fc1")

        pool2_flat = tf.reshape(pool2, [-1, 11*11*64])
        fc1 = tf.nn.relu(tf.add(tf.matmul(pool2_flat, w_fc1), b_fc1))

        keep_prob = tf.placeholder(tf.float32, name = "keep_prob")
        fc1_drop = tf.nn.dropout(fc1, keep_prob)

    with tf.name_scope('fc2'):
        w_fc2 = tf.Variable(tf.truncated_normal([1536, 10], stddev = 0.1), name="w_fc2")
        b_fc2 = tf.Variable(tf.truncated_normal([10], stddev = 0.1), name="b_fc2")

        y = tf.softmax(tf.add(tf.matmul(fc1_drop, w_fc2), b_fc2))
    
    with tf.Session() as sess:
        output = sess.run(y, feed_dict = {x : images, keep_prob : 1.0})
        
    return output

In [ ]:
labels = CNNClassifier(digits)
for i in range(81):
    if np.max(labels[i]) > 0.8:
        labels[i] = np.argmax(labels[i])
    else:
        labels[i] = 0
grid = np.reshape(labels, 9, 9)

print(grid)

In [ ]:
def nextLocation(grid, x, y):
    if x == 9:
        return 9, 9
    i = y + 1
    while i < 9:
        if grid[x, y] is 0:
            return x, i
        i = i + 1
    return nextLocation(grid,  x, y)
    
    
def isSafe(grid, x, y, num):
    for i in range(9):
        if i is not y and grid[x, i] is num:
            return False
    for i in range(9):
        if i is not x and grid[i, x] is num:
            return False
    for i in range(3 * (x // 3), 3 * (x // 3) + 3, 1):
        for j in range(3 * (y // 3), 3 * (y // 3) + 3, 1):
            if i is not x and j is not y and grid[i, j] is num:
                return False
    return True


def solveSudoku(grid, x, y):
    if x == 9:
        return True
    p, q = nextLocation(grid, x, y)
    for i in range(1, 10, 1):
        if isSafe(grid, x, y, i):
            grid[x, y] = i
            if solveSudoku(grid, p, q):
                return True
            grid[x, y] = 0
    return False


def printSudoku(grid):
    for i in range(9):
        for j in range(9):
            if (j + 1) % 3 is not 0:
                print(grid[i, j], end = ' ')
            else:
                print(grid[i, j], end = ' | ')
        print()
        
        if (i + 1) % 3 is 0:
            for j in range(9):
                print('-', end = ' ')
            print()

In [ ]:
if solveSudoku(grid) is False:
    print("Unable to solve Sudoku")
else:
    printSudoku(grid)